#### `model.json(...)`

The `.json()` method will serialise a model to JSON. (For models with a `custom root type`, only the value for the `__root__` key is serialised)

Arguments:

* `include`: fields to include in the returned dictionary; see below

* `exclude`: fields to exclude from the returned dictionary; see below

* `by_alias`: whether field aliases should be used as keys in the returned dictionary; default `False`

* `exclude_unset`: whether fields which were not set when creating the model and have their 
default values should be excluded from the returned dictionary; default `False`

* `exclude_defaults`: whether fields which are equal to their default values (whether set or otherwise) should be excluded from the returned dictionary; default `False`

* `exclude_none`: whether fields which are equal to `None` should be excluded from the returned dictionary; default `False`

* encoder: a custom encoder function passed to the `default` argument of `json.dumps()`; defaults to a custom encoder designed to take care of all common types

* `**dumps_kwargs`: any other keyword arguments are passed to `json.dumps()`, e.g. `indent`.

_pydantic_ can serialise many commonly used types to JSON (e.g. `datetime`, `date` or `UUID`) which would normally fail with a simple `json.dumps(foobar)`.

In [1]:
import ujson
import orjson
from pydantic import BaseModel
from typing import List, Optional
from pydantic.json import timedelta_isoformat
from pydantic.validators import int_validator
from datetime import date, datetime, timedelta

In [2]:
class BarModel(BaseModel):
    whatever: int

In [3]:
class FooBarModel(BaseModel):
    foo: datetime
    bar: BarModel

In [4]:
m = FooBarModel(foo=datetime(2032, 6, 1, 12, 13, 14), bar={"whatever": 123})
print(f"{m.json() = }")

m.json() = '{"foo": "2032-06-01T12:13:14", "bar": {"whatever": 123}}'


##### `json_encoders`

Serialisation can be customised on a model using the `json_encoders` config property; the keys should be types (or names of types for forward references), and the values should be functions which serialise that type.

In [5]:
class WithCustomEncoders(BaseModel):
    dt: datetime
    diff: timedelta

    class Config:
        json_encoders = {
            datetime: lambda v: v.timestamp(),
            timedelta: timedelta_isoformat,
        }

In [6]:
m = WithCustomEncoders(dt=datetime(2032, 6, 1), diff=timedelta(hours=100))
print(f"{m.json() = }")

m.json() = '{"dt": 1969639200.0, "diff": "P4DT4H0M0.000000S"}'


By default, `timedelta` is encoded as a simple float of total seconds. The `timedelta_isoformat` is provided as an optional alternative which implements `ISO 8601` time diff encoding.

The `json_encoders` are also merged during the models inheritance with the child encoders taking precedence over the parent one.

In [7]:
class BaseClassWithEncoders(BaseModel):
    dt: datetime
    diff: timedelta

    class Config:
        json_encoders = {
            datetime: lambda v: v.timestamp()
        }

In [8]:
class ChildClassWithEncoders(BaseClassWithEncoders):
    class Config:
        json_encoders = {
            timedelta: timedelta_isoformat
        }

In [9]:
m = ChildClassWithEncoders(dt=datetime(2032, 6, 1), diff=timedelta(hours=100))
print(f"{m.json() = }")

m.json() = '{"dt": 1969639200.0, "diff": "P4DT4H0M0.000000S"}'


##### Serialising self-reference or other models

By default, models are serialised as dictionaries. If you want to serialise them differently, you can add `models_as_dict=False` when calling `json()` method and add the classes of the model in `json_encoders`. In case of forward references, you can use a string with the class name instead of the class itself.

In [10]:
class Address(BaseModel):
    city: str
    country: str

In [11]:
class User(BaseModel):
    name: str
    address: Address
    friends: Optional[List["User"]] = None

    class Config:
        json_encoders = {
            Address: lambda a: f"{a.city} ({a.country})",
            "User": lambda u: f"{u.name} in {u.address.city} "
                f"({u.address.country[:2].upper()})",
        }

In [12]:
User.update_forward_refs()

In [13]:
wolfgang = User(
    name="Wolfgang",
    address=Address(city="Berlin", country="Deutschland"),
    friends=[
        User(name="Pierre", address=Address(city="Paris", country="France")),
        User(name="John", address=Address(city="London", country="UK")),
    ],
)
print(f"{wolfgang.json(models_as_dict=False) = }")

wolfgang.json(models_as_dict=False) = '{"name": "Wolfgang", "address": "Berlin (Deutschland)", "friends": ["Pierre in Paris (FR)", "John in London (UK)"]}'


##### Serialising subclasses

Subclasses of common types are automatically encoded like their super-classes.

In [14]:
class DayThisYear(date):
    """
    Contrived example of a special type of date that
    takes an int and interprets it as a day in the current year
    """

    @classmethod
    def __get_validators__(cls):
        yield int_validator
        yield cls.validate

    @classmethod
    def validate(cls, v: int):
        return date.today().replace(month=1, day=1) + timedelta(days=v)

In [15]:
class FooModel(BaseModel):
    date: DayThisYear

In [16]:
m = FooModel(date=300)
print(f"{m.json() = }")

m.json() = '{"date": "2023-10-28"}'


##### Custom JSON (de)serialisation

To improve the performance of encoding and decoding JSON, alternative JSON implementations (e.g. `ujson`) can be used via the `json_loads` and `json_dumps` properties of `Config`.

In [17]:
class UjsonUser(BaseModel):
    id: int
    name = "John Doe"
    signup_ts: datetime = None

    class Config:
        json_loads = ujson.loads

In [18]:
user = UjsonUser.parse_raw('{"id": 123,"signup_ts":1234567890,"name":"John Doe"}')
print(f"{user = }")

user = UjsonUser(id=123, signup_ts=datetime.datetime(2009, 2, 13, 23, 31, 30, tzinfo=datetime.timezone.utc), name='John Doe')


`ujson` generally cannot be used to dump JSON since it doesn't support encoding of objects like datetimes and does not accept a `default` fallback function argument. To do this, you may use another library like `orjson`.

In [19]:
def orjson_dumps(v, *, default):
    # orjson.dumps returns bytes, to match standard json.dumps we need to decode
    return orjson.dumps(v, default=default).decode()

In [20]:
class OrjsonUser(BaseModel):
    id: int
    name = "John Doe"
    signup_ts: datetime = None

    class Config:
        json_loads = orjson.loads
        json_dumps = orjson_dumps

In [21]:
user = OrjsonUser.parse_raw('{"id":123,"signup_ts":1234567890,"name":"John Doe"}')
print(f"{user.json() = }")

user.json() = '{"id":123,"signup_ts":"2009-02-13T23:31:30+00:00","name":"John Doe"}'


Note that `orjson` takes care of `datetime` encoding natively, making it faster than `json.dumps` but meaning you cannot always customise the encoding using `Config.json_encoders`.